In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
import os

# Parameters
m = 1
v = 10
N_part = 625
m_disk = 10
R_disk = 10
L = 260
sigma = 1
epsilon = 1
T_tot = 10
dt = 0.005
trajectory = []
msd_list = []

# Tkinter visualization setup
window = tk.Tk()
canvas = tk.Canvas(window, width=L, height=L, bg="white")
canvas.pack()


# Initialize disk
disk = {'x': L / 2, 'y': L / 2, 'vx': 0, 'vy': 0}
disk_obj = canvas.create_oval(disk['x'] - R_disk, disk['y'] - R_disk,
                              disk['x'] + R_disk, disk['y'] + R_disk, fill="black")

# Initialize particles on a grid
particles = []
particle_objs = []
for i in range(int(np.sqrt(N_part))):
    for j in range(int(np.sqrt(N_part))):
        x = i * L / np.sqrt(N_part) + L / (2 * np.sqrt(N_part))
        y = j * L / np.sqrt(N_part) + L / (2 * np.sqrt(N_part))
        angle = np.random.uniform(0, 2 * np.pi)
        vx = v * np.cos(angle)
        vy = v * np.sin(angle)
        particle = {'x': x, 'y': y, 'vx': vx, 'vy': vy}
        particles.append(particle)
        particle_obj = canvas.create_oval(x - 2, y - 2, x + 2, y + 2, fill="blue")
        particle_objs.append(particle_obj)

# Remove particles within the disk radius plus 3*sigma
particles = [p for p in particles if np.sqrt((p['x'] - disk['x']) ** 2 + (p['y'] - disk['y']) ** 2) > (R_disk + 3 * sigma)]




In [ ]:
# Plot trajectory and MSD
def plot_results():
    # Plot disk's trajectory
    trajectory_x, trajectory_y = zip(*trajectory)
    plt.figure(figsize=(6, 5))  # 创建单独的图窗口
    plt.plot(trajectory_x, trajectory_y, color='blue')
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.title("Disk Trajectory in Cartesian Plane")
    plt.grid(True)
    plt.savefig(os.path.join(output_folder, 'disk_trajectory.png'))
    plt.show()  
    plt.close() 

    # MSD Plot
    plt.figure(figsize=(6, 5))  # 创建另一个单独的图窗口
    time = np.arange(1, len(msd_list) + 1) * dt
    plt.plot(time, msd_list, color='red')
    plt.xlabel("Time (t)")
    plt.ylabel("Mean Square Displacement (MSD)")
    plt.title("Mean Square Displacement over Time")
    plt.grid(True)
    plt.savefig(os.path.join(output_folder, 'msd_over_time.png'))
    plt.show()  # 显示第二张图
    plt.close()

# Lennard-Jones force calculation
def lennard_jones_force(disk, particle):
    dx = disk['x'] - particle['x']
    dy = disk['y'] - particle['y']
    r = np.sqrt(dx ** 2 + dy ** 2) - R_disk
    if r < 3 * sigma:
        F = 24 * epsilon * ((2 * (sigma / r) ** 12) - (sigma / r) ** 6) / r
        fx = F * dx / r
        fy = F * dy / r
        return fx, fy
    return 0, 0

# Initialize lists for trajectory and MSD calculations
trajectory = []
msd_list = []
time_steps = int(T_tot / dt)

# Update positions and velocities
def update_positions(step=0):
    
    # Record disk's position for trajectory and MSD calculation
    trajectory.append((disk['x'], disk['y']))

    # Calculate MSD at each step
    if step > 0:
        initial_x, initial_y = trajectory[0]
        msd = (disk['x'] - initial_x) ** 2 + (disk['y'] - initial_y) ** 2
        msd_list.append(msd)

    # Update disk position
    disk_fx, disk_fy = 0, 0
    for i, particle in enumerate(particles):
        fx, fy = lennard_jones_force(disk, particle)
        disk_fx += fx
        disk_fy += fy
        particle['vx'] += -fx / m * dt
        particle['vy'] += -fy / m * dt
        particle['x'] += particle['vx'] * dt
        particle['y'] += particle['vy'] * dt

        # Boundary conditions for particles (reflecting)
        if particle['x'] <= 0 or particle['x'] >= L:
            particle['vx'] *= -1
        if particle['y'] <= 0 or particle['y'] >= L:
            particle['vy'] *= -1

        # Update particle position in Tkinter
        canvas.coords(particle_objs[i], particle['x'] - 2, particle['y'] - 2,
                      particle['x'] + 2, particle['y'] + 2)

    # Update disk velocity and position based on net force
    disk['vx'] += disk_fx / m_disk * dt
    disk['vy'] += disk_fy / m_disk * dt
    disk['x'] += disk['vx'] * dt
    disk['y'] += disk['vy'] * dt

    # Boundary conditions for the disk (reflecting)
    if disk['x'] <= R_disk or disk['x'] >= L - R_disk:
        disk['vx'] *= -1
    if disk['y'] <= R_disk or disk['y'] >= L - R_disk:
        disk['vy'] *= -1

    # Update disk position in Tkinter
    canvas.coords(disk_obj, disk['x'] - R_disk, disk['y'] - R_disk, disk['x'] + R_disk, disk['y'] + R_disk)
    canvas.create_oval(disk['x'] - 0.2, disk['y'] - 0.2, disk['x'] + 0.2, disk['y'] + 0.2, fill="red", outline="red")

    if step % 100 ==0:
        print(f'time step is {step}, disk position is {disk['x']}, {disk['y']}')
    # Repeat the update
    if step < time_steps:
        window.after(int(dt * 1000), lambda: update_positions(step + 1))
    else:
        plot_results()
        running = False

# Start simulation
update_positions()
window.mainloop()


